In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=1,
    )

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
)

examples = [
    {
        "question": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human. Always respond using exactly three emojis. Do not use any text."),
        MessagesPlaceholder(variable_name="history"),
        few_shot_prompt,
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

invoke_chain("매트릭스")
invoke_chain("인터스텔라")

💊🕶️💻
🚀👨‍🚀🌌


In [2]:
invoke_chain("Tell me which movie I asked about first.")

💊🕶️💻
